In [70]:
from pyspark import SparkContext, SparkConf

In [71]:
from operator import add

In [72]:
conf = SparkConf().setMaster("local[*]")
sc = SparkContext.getOrCreate()

In [73]:
def rdd_to_dictionary(rdd):
    return sc.parallelize(rdd).collectAsMap()

In [74]:
def write_json(dictionary,filename):
    import json  

    with open(filename, "w") as outfile:  
        json.dump(dictionary, outfile) 
    

In [75]:
data_bag = spark.read.text(["data/4300-0.txt","data/pg19033.txt", "data/pg36.txt","data/pg514.txt","data/pg1497.txt","data/pg3207.txt","data/pg42671.txt","data/pg6130.txt"]).rdd.map(lambda r: r[0])

In [76]:
words_count = data_bag.flatMap(lambda x: x.split(' '))\
                .map(lambda x: x.lower()) \
                .map(lambda x: (x, 1))\
                .reduceByKey(add)\
                .filter(lambda x: x[1] > 1)\
                .filter(lambda x: len(x[0]) > 0)

In [77]:
top_40_words = words_count.takeOrdered(40, key = lambda x: -x[1])
top_40_words_dictionary = rdd_to_dictionary(top_40_words)

In [79]:
write_json(top_40_words_dictionary,"sp1.json")

In [80]:
stop_words_data_bag = spark.read.text("data/stopwords.txt").rdd.map(lambda r: r[0])
stop_words = stop_words_data_bag.flatMap(lambda x: x.split('\n'))\
                .map(lambda x: x.lower()).map(lambda x: (x, 1)).reduceByKey(add)
# print(stop_words.collect())

In [81]:
top_40_words_without_stop_words = words_count.subtractByKey(stop_words).takeOrdered(40, key = lambda x: -x[1])

# print(top_40_words_without_stop_words)

In [82]:
top_40_words_without_stop_words_dictionary = rdd_to_dictionary(top_40_words_without_stop_words)
# print(top_40_words_without_stop_words_dictionary)

In [83]:
write_json(top_40_words_without_stop_words_dictionary,"sp2.json")

In [9]:
sc.stop()